In [326]:
import re
# !pip install tweepy
import tweepy 
import nltk
import numpy as np
from nltk.tokenize import word_tokenize


In [4]:
# !pip install textBlob

In [327]:
from tweepy import OAuthHandler
from textblob import TextBlob

In [328]:
# Twitter API access: 

consumer_key = '9RkzAwLLv1DoPcZBbQhh5aPoo'
consumer_key_sec = 'hYuceLAy4VEgRauLElI9ug1apCl0GeMXTCBHgbtZTP3bTeWyTv'
access_token = '1082104728-67g0WHv3Uc61bi6a9SunusdLcHOsZxHtFONKryH'
access_token_secret = 'EP3k3ywagtGk0CYU2cOASqSq5LQbeYjXneazC7TgqVO9h'


In [329]:
# Connect twitter with api and do the authentication: 
auth = tweepy.OAuthHandler(consumer_key, consumer_key_sec)
auth.set_access_token(access_token, access_token_secret)

# Access Twitter API 
api = tweepy.API(auth, wait_on_rate_limit=True)
api

In [330]:
# Obtaining tweets based on the Topic
tweets = api.search('Tanishq', count=100, lang = 'en')
df = pd.DataFrame(columns=['text', 'created_at', 'name','rt'])
for tweet in tweets:
    curr_row = {'text': tweet.text,
                'created_at': tweet.created_at,
                'name': tweet.user.name, 
               'rt': tweet.retweet_count}
    df = df.append(curr_row, ignore_index=True)

In [331]:
df.shape

(100, 4)

In [332]:
df.head() 

,text,created_at,name,rt
0,RT @lakshmianand96: If #tanishq got spooked by...,2020-10-16 09:09:33,Anti Sickular,1
1,RT @sardesairajdeep: Suprabhatam! Friday comme...,2020-10-16 09:09:28,dv.sharma🇮🇳,529
2,RT @kunalkamra88: Everyone who is offended wit...,2020-10-16 09:09:16,Rahul,7791
3,RT @Memeghnad: TATA will continue to give fund...,2020-10-16 09:09:15,Boy_Stuck_In_Limbo,244
4,RT @ishkarnBHANDARI: So it has come out NDTV d...,2020-10-16 09:09:15,Anu,5682


In [333]:
# Sentiment Analyser: 
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti = SentimentIntensityAnalyzer() 
senti.polarity_scores('i hate apple')

{'neg': 0.787, 'neu': 0.213, 'pos': 0.0, 'compound': -0.5719}

In [334]:
df['sentiment_vader_text'] = df['text'].apply(lambda v: senti.polarity_scores(v)['compound'])

In [335]:
df.head()

,text,created_at,name,rt,sentiment_vader_text
0,RT @lakshmianand96: If #tanishq got spooked by...,2020-10-16 09:09:33,Anti Sickular,1,-0.5719
1,RT @sardesairajdeep: Suprabhatam! Friday comme...,2020-10-16 09:09:28,dv.sharma🇮🇳,529,0.3595
2,RT @kunalkamra88: Everyone who is offended wit...,2020-10-16 09:09:16,Rahul,7791,-0.2500
3,RT @Memeghnad: TATA will continue to give fund...,2020-10-16 09:09:15,Boy_Stuck_In_Limbo,244,-0.5106
4,RT @ishkarnBHANDARI: So it has come out NDTV d...,2020-10-16 09:09:15,Anu,5682,-0.4767


In [336]:
# stop_words_all = list(stop_words_all)
# stop_words_all.extend(['rt','RT'])
# stop_words_all

# stop words: 
custom_stop_words = ['rt','RT']
common_stop_words = nltk.corpus.stopwords.words('english')
stop_words_all = np.hstack([custom_stop_words, common_stop_words])
stop_words_all = list(stop_words_all)

In [337]:
docs_t = df['text'].str.lower()
docs_t = docs_t.str.replace('[^a-z ]', '')
docs_t = docs_t.str.split(' ')
words_rows = docs_t.tolist()

docs_clean_new = []
for row in words_rows:
    row_words = [word for word in row if word not in stop_words_all]  
    docs_clean_new.append(' '.join(row_words))
docs_clean_new

['lakshmianand tanishq got spooked market cap fall negative reaction potential customers coz bigoted ad',
 'sardesairajdeep suprabhatam friday comment piece  tatas could learn hamara bajaj read share like https',
 'kunalkamra everyone offended tanishq advertisement please also boycott dowry ',
 'memeghnad tata continue give funds bjp despite whole tanishq fiascoyou know whybecause country laws',
 'ishkarnbhandari come ndtv fake news attack tanishq storeyears change viewership goes away ndtv agenda',
 'freethinker morphed tweet portraying shefali vaidya passing imitation jewellery necklace set tanishq shared mo',
 'liberals logic single source burnt whole city one fb post still peaceful hindus protest httpstcoeauoqctnz',
 'grihaatul im married muslim household respects accepts religious identity tanishq advert wasnt figment',
 'byrakeshsimha  million white british children victims muslim rape gangs says cbn news hindu sikh girls also target',
 'shripadmanthan divyadutta tanishq ad withd

In [338]:
# Lets add the clean text as a column: 
# First convert list to series: 
column_val = pd.Series(docs_clean_new)

# Using insert function add the col
df.insert(loc = 2, column = 'clean_text', value = column_val)

In [339]:
df['sentiment_vader_cleantext'] = df['clean_text'].apply(lambda v: senti.polarity_scores(v)['compound'])

In [340]:
df.head() 

,text,created_at,clean_text,name,rt,sentiment_vader_text,sentiment_vader_cleantext
0,RT @lakshmianand96: If #tanishq got spooked by...,2020-10-16 09:09:33,lakshmianand tanishq got spooked market cap fa...,Anti Sickular,1,-0.5719,-0.5719
1,RT @sardesairajdeep: Suprabhatam! Friday comme...,2020-10-16 09:09:28,sardesairajdeep suprabhatam friday comment pie...,dv.sharma🇮🇳,529,0.3595,0.5719
2,RT @kunalkamra88: Everyone who is offended wit...,2020-10-16 09:09:16,kunalkamra everyone offended tanishq advertise...,Rahul,7791,-0.2500,-0.2500
3,RT @Memeghnad: TATA will continue to give fund...,2020-10-16 09:09:15,memeghnad tata continue give funds bjp despite...,Boy_Stuck_In_Limbo,244,-0.5106,0.0000
4,RT @ishkarnBHANDARI: So it has come out NDTV d...,2020-10-16 09:09:15,ishkarnbhandari come ndtv fake news attack tan...,Anu,5682,-0.4767,-0.7351


In [344]:
def assign_sentiment(v):
    if v > 0.25:
        return 'Positive'
    elif v < -0.25:
        return 'Negative'
    else:
        return 'Neutral'
df['sentiment_vader_cleantext'] = df['sentiment_vader_cleantext'].apply(assign_sentiment)
df['sentiment_vader_cleantext'].value_counts()

Neutral     51
Positive    33
Negative    16
Name: sentiment_vader_cleantext, dtype: int64

## Supervised models 

In [248]:
#df.columns

custom_stop_words = ['rt','RT']
common_stop_words = nltk.corpus.stopwords.words('english')
stop_words_all = np.hstack([custom_stop_words, common_stop_words])
len(stop_words_all)

from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
stemmer = PorterStemmer()

In [252]:
docs_new = df['text'].str.lower()
docs_new = docs_new.str.replace('[^a-z#@ ]', '')
docs_new = docs_new.str.split(' ')
words_rows = docs_new.tolist()

docs_clean_new = []
for row in words_rows:
    row_words = [stemmer.stem(word) for word in row if word not in stop_words_all]  
    docs_clean_new.append(' '.join(row_words))

model_new = CountVectorizer(vocabulary=model_dtm.vocabulary_)
dtm_new = pd.DataFrame(model_new.fit_transform(docs_clean_new).toarray(),
                      columns=model_dtm.get_feature_names())

In [256]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier()
model_rf.fit(train_x, train_y)

NameError: name 'train_y' is not defined

In [255]:
len(model_rf.predict(dtm_new))

NameError: name 'model_rf' is not defined

In [257]:
query = input('What topic?: ')
number = int(input('how many tweets?: '))
results = api.search(lang = 'en', q = query + '-rt', count = number, result_type = 'recent')



What topic?: Apple 
how many tweets?: 100


In [258]:
results 

[Status(_api=<tweepy.api.API object at 0x000001C4BABFC100>, _json={'created_at': 'Fri Oct 16 08:37:44 +0000 2020', 'id': 1317021900145631234, 'id_str': '1317021900145631234', 'text': 'Wedbush Increases Apple $AAPL Price Target to $150.00 https://t.co/qb6iaKDYDi', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [{'text': 'AAPL', 'indices': [24, 29]}], 'user_mentions': [], 'urls': [{'url': 'https://t.co/qb6iaKDYDi', 'expanded_url': 'http://thelincolnianonline.com/?p=4864961', 'display_url': 'thelincolnianonline.com/?p=4864961', 'indices': [54, 77]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://zapier.com/" rel="nofollow">Zapier.com</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 930803511918411776, 'id_str': '930803511918411776', 'name': 'The Lincolnian', 'screen_name': 'ThisLincolnian', 'locat